<a href="https://colab.research.google.com/github/github-rokon/CVPR/blob/main/Face_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pillow pyheif

In [2]:
import tensorflow as tf
from tensorflow import keras
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'


import pyheif
import cv2
import random
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from PIL import Image


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
dataset_path = '/content/drive/MyDrive/Colab Notebooks/Dataset'

In [5]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [6]:
def load_heic_image(image_path):
    # Read HEIC image using pyheif
    heif_file = pyheif.read(image_path)
    # Convert it into a format that Pillow can work with
    image = Image.frombytes(
        heif_file.mode,
        heif_file.size,
        heif_file.data,
        "raw",
        heif_file.mode,
        heif_file.stride,
    )
    return image

In [12]:
# Load dataset with resizing images to the same dimensions
def load_dataset(dataset_path):
    images = []
    labels = []
    label_names = os.listdir(dataset_path)

    for index, name in enumerate(tqdm(label_names)):
        folder_path = os.path.join(dataset_path, name)
        for image_name in os.listdir(folder_path):
            image_path = os.path.join(folder_path, image_name)

            # Check if the file is a HEIC or regular image
            if image_name.lower().endswith('.heic'):
                try:
                    image = load_heic_image(image_path)  # Load HEIC image
                    image = np.array(image)
                except Exception as e:
                    print(f"Error loading HEIC image: {image_path} - {e}")
                    continue
            else:
                image = cv2.imread(image_path)  # Load non-HEIC image with OpenCV

            if image is not None:  # Check if the image was successfully loaded
                image = cv2.resize(image, (224, 224))  # Resize to 224x224
                images.append(image)
                labels.append(index)
            else:
                print(f"Warning: Could not load image {image_path}")

    images = np.array(images, dtype='float32') / 255.0  # Normalize
    labels = np.array(labels)
    labels = to_categorical(labels, num_classes=len(label_names))

    return images, labels, label_names # Return the images and labels

# Example usage
dataset_path = '/content/drive/MyDrive/Colab Notebooks/Dataset' # Replace with the actual path to your dataset
images, labels, label_names = load_dataset(dataset_path) # Call the function and assign the returned values

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

  2%|▏         | 1/56 [00:00<00:40,  1.34it/s]

 80%|████████  | 45/56 [00:41<00:03,  3.38it/s]

Error loading HEIC image: /content/drive/MyDrive/Colab Notebooks/Dataset/SUN/IMG_6943.HEIC - Input is not a HEIF/AVIF file


100%|██████████| 56/56 [00:49<00:00,  1.13it/s]


In [13]:


# Test different camera indices
for i in range(5):  # Try indices 0 to 4
    cap = cv2.VideoCapture(i)
    if cap.isOpened():
        print(f"Camera found at index {i}")
        cap.release()

In [14]:
# Build CNN model
def build_model(num_classes):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.Dense(num_classes, activation='softmax'),
    ])
    return model

model = build_model(num_classes=len(label_names))
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [15]:
# Train model
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test), batch_size=32)

Epoch 1/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 82s 7s/step - accuracy: 0.0244 - loss: 4.3080 - val_accuracy: 0.0575 - val_loss: 3.9558
Epoch 2/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 78s 7s/step - accuracy: 0.1360 - loss: 3.8091 - val_accuracy: 0.1264 - val_loss: 3.9029
Epoch 3/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 82s 7s/step - accuracy: 0.2493 - loss: 3.4859 - val_accuracy: 0.1839 - val_loss: 3.7307
Epoch 4/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 82s 7s/step - accuracy: 0.4620 - loss: 2.7714 - val_accuracy: 0.1839 - val_loss: 3.5732
Epoch 5/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 82s 7s/step - accuracy: 0.6377 - loss: 2.0840 - val_accuracy: 0.2184 - val_loss: 3.2854
Epoch 6/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 73s 7s/step - accuracy: 0.7726 - loss: 1.3700 - val_accuracy: 0.3448 - val_loss: 2.8462
Epoch 7/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 76s 7s/step - accuracy: 0.9204 - loss: 0.6412 - val_accuracy: 0.2644 - val_loss: 3.2362
Epoch 8/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 82s 7s/step - accuracy: 0.9405 - loss: 0.4089 - val_accuracy: 0.3793 - val_loss:

In [17]:


def real_time_recognition(model, label_names):
    # Initialize the webcam (use 0 as the parameter to select the default webcam)
    cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)  # For Windows

    # Check if the webcam is opened correctly
    if not cap.isOpened():
        raise IOError("Cannot open webcam")



while True:
    ret, frame = cap.read()  # Read a frame from the webcam
    if not ret:
        break

    # Convert the frame to grayscale (Haar cascades work with grayscale images)
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        # Draw a rectangle around each face
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

        # Extract the face region
        face_region = frame[y:y+h, x:x+w]

        # Preprocess the face region for prediction
        face = cv2.resize(face_region, (224, 224))
        face = np.expand_dims(face, axis=0)
        face = face / 255.0

        # Make prediction
        prediction = model.predict(face)
        predicted_class = label_names[np.argmax(prediction)]

        # Display the predicted class label
        cv2.putText(frame, predicted_class, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Show the frame with face rectangles and predicted class labels
    cv2.imshow('Real-time Face Recognition', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


    # Release the webcam and close all OpenCV windows
    cap.release()
    cv2.destroyAllWindows()

# Assuming 'model' is your trained model and 'label_names' is a list of class names
# real_time_recognition(model, label_names)